In [ ]:
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
import pandas as pd
import pandas_flavor as pf
from vfu_package.datapipeline import *

In [ ]:
datapipe.service = 'vfu'
datapipe.runtime = 'local'

### Items

In [ ]:
@pf.register_dataframe_method
def step_consolidate_item_learning_levels(df, col, mapping=None):
    if mapping:
        learning_level_names = mapping
    else:
        learning_level_names = {'1': '1:Core', '2':'2:Intermediate', '3':'3:Advanced',
                    '4':'4:Expert', 'SCM Essentials':'1:Core', '5':'5:Others'}
    df[col] = df[col].str.split(' ').str[0]
    df[col] = np.where(df[col] == '', 'Missing Info', df[col])
    df[col] = df[col].map(learning_level_names)  
    df[col] = df[col].fillna(value='Missing Info')
    return df

In [ ]:
datapipe.entity = 'assets'

#### loading raw data:

In [ ]:
items_raw = (pd.DataFrame()
             .step_load_entity_definition()
             .step_load_raw_data()
             .step_rename_columns()
            )

#### generating raw data profile report

In [ ]:
#items_raw_profile_report = datapipe.generate_profile_report(df=items_raw, mode='Raw')

#### check if any invalid data:

In [ ]:
items_invalid = (items_raw
                 .step_replace_missing_values_with_nan()
                 .step_validate_data()
                 .step_store_invalid_data()
                )

#### cleaning data:

In [ ]:
items_raw.modified

In [ ]:
items_cleaned = (items_raw
                 .step_replace_missing_values_with_nan()
                 .step_remove_not_needed_columns()
                 .step_remove_rows_with_invalid_ids()
                 .step_remove_duplicate_rows()
                 #.step_format_dates(cols=['created'], date_format='%m/%d/%Y')
                 .step_format_dates(cols=['created'])
                 #.step_format_dates(cols=['modified'], date_format='%Y-%m-%d')
                 .step_format_dates(cols=['modified'])
                 .step_format_round_numeric_column(col='average_rating', decimal_places=2)
                 .step_change_datatypes()
                 .step_consolidate_item_learning_levels(col='learning_level', mapping=None)
                 .step_store_cleaned_data()
                )

#### generating profile report of cleaned data

In [ ]:
items_cleaned_profile_report = datapipe.generate_profile_report(df=items_cleaned,
                                                                mode='Cleaned', 
                                                                save_to='../data/dataquality/')

In [ ]:
items_academy_df = pd.DataFrame(items_cleaned.academy.unique(), columns=['Current Academy Name'])
items_academy_df['Consolidated Academy Name'] = None
items_academy_df.to_excel('../data/dataquality/vfu-streamline-academy-names.xlsx')

### nps

In [ ]:
datapipe.entity = 'nps'

#### loading raw data:

In [ ]:
nps_raw = (pd.DataFrame()
             .step_load_entity_definition()
             .step_load_raw_data()
             .step_rename_columns()
            )

#### generating profile report of raw data

In [ ]:
nps_raw_profile_report = datapipe.generate_profile_report(df=nps_raw, mode='Raw')

#### check if any invalid data:

In [ ]:
nps_invalid = (nps_raw
                 .step_replace_missing_values_with_nan()
                 .step_validate_data()
                 .step_store_invalid_data()
                )

#### cleaning data:

In [ ]:
nps_cleaned = (nps_raw
                 .step_replace_missing_values_with_nan()
                 .step_remove_not_needed_columns()
                 .step_remove_rows_with_invalid_ids()
                 .step_remove_duplicate_rows()
                 .step_format_dates(cols=['modified'])
                 .step_format_dates(cols=['created'])
                 .step_change_datatypes()
                 .step_store_cleaned_data()
                )

#### generating profile report of cleaned data

In [ ]:
nps_cleaned_profile_report = datapipe.generate_profile_report(df=nps_cleaned, mode='Cleaned')

### lms-asset-events 
* data about items completed or not completed
* in progress and assigned data is currently provided in separate files

#### lms-asset-events specific functions:

In [ ]:
@pf.register_dataframe_method
def step_consolidate_lms_item_assignment_types(df, col, mapping=None):
    
    if mapping:
        assignment_type_names = mapping
    else:
        assignment_type_names = {'OPT': 'Optional', 'REC':'Recommended', 'REQ':'Mandatory'}
    df[col] = df[col].str.split(' ').str[0]
    df[col] = np.where(df[col] == '', 'Optional', df[col])
    df[col] = df[col].map(assignment_type_names)  
    df[col] = df[col].fillna(value='Optional')

    return df

In [ ]:
datapipe.entity = 'lms-asset-events'

#### loading raw data:

In [ ]:
pd.DataFrame().step_load_entity_definition()

In [ ]:
lms_items_completed_raw = (pd.DataFrame()
             .step_load_entity_definition()
             .step_load_raw_data()
             .step_rename_columns()
            )
#lms_items_completed_raw_profile_report = datapipe.generate_profile_report(df=items_raw, mode='Raw')

#### generating profile report of raw data

#### check if any invalid data:

In [ ]:
lms_items_completed_invalid = (lms_items_completed_raw
                               .step_replace_missing_values_with_nan()
                               .step_validate_data()
                               .step_store_invalid_data()
                              )

#### cleaning data:

In [ ]:
lms_items_completed_cleaned = (lms_items_completed_raw
                               .step_replace_missing_values_with_nan()
                               .step_remove_not_needed_columns()
                               .step_remove_rows_with_invalid_ids()
                               .step_remove_duplicate_rows()
                               .step_format_dates(cols=['completion_date'])
                               .step_consolidate_lms_item_assignment_types(col='assignment_type')
                               .step_change_datatypes()
                               .step_store_cleaned_data()
                              )

In [ ]:
lms_items_completed_cleaned.item_title.str.contains('DRW').sum()

#### generating profile report of cleaned data

### page hits

In [ ]:
datapipe.entity = 'page-hits'

In [ ]:
page_hits_raw = (pd.DataFrame()
                 .step_load_entity_definition()
                 .step_load_raw_data()
                 .step_rename_columns()
                )

In [ ]:
page_hits_invalid = (page_hits_raw
                     .step_replace_missing_values_with_nan()
                     .step_validate_data()
                     .step_store_invalid_data()
                    )

In [ ]:
page_hits_cleaned = (page_hits_raw
                     .step_replace_missing_values_with_nan()
                     .step_remove_not_needed_columns()
                     .step_remove_rows_with_invalid_ids()
                     .step_remove_duplicate_rows()
                     .step_change_datatypes()
                     .step_store_cleaned_data()
                )

### global-navigation

In [ ]:
datapipe.entity = 'global-navigation'

In [ ]:
global_navigation_raw = (pd.DataFrame()
                         .step_load_entity_definition()
                         .step_load_raw_data()
                         .step_rename_columns()
                        )

In [ ]:
global_navigation_invalid = (global_navigation_raw
                             .step_replace_missing_values_with_nan()
                             .step_validate_data()
                             .step_store_invalid_data()
                            )

In [ ]:
global_navigation_cleaned = (global_navigation_raw
                             .step_replace_missing_values_with_nan()
                             .step_remove_not_needed_columns()
                             .step_remove_rows_with_invalid_ids()
                             .step_remove_duplicate_rows()
                             .step_change_datatypes()
                             .step_store_cleaned_data()
                            )

In [ ]:
global_navigation_cleaned.head()

### search-page

In [ ]:
datapipe.entity = 'search-page'

In [ ]:
search_page_raw = (pd.DataFrame()
                   .step_load_entity_definition()
                   .step_load_raw_data()
                   .step_rename_columns()
                  )

In [ ]:
search_page_invalid = (search_page_raw
                       .step_replace_missing_values_with_nan()
                       .step_validate_data()
                       .step_store_invalid_data()
                      )

In [ ]:
search_page_cleaned = (search_page_raw
                       .step_replace_missing_values_with_nan()
                       .step_remove_not_needed_columns()
                       .step_remove_rows_with_invalid_ids()
                       .step_remove_duplicate_rows()
                       .step_change_datatypes()
                       .step_store_cleaned_data()
                      )

### of-course-me-launches

In [ ]:
datapipe.entity = 'of-course-me-launches'

In [ ]:
of_course_me_launches_raw = (pd.DataFrame()
                             .step_load_entity_definition()
                             .step_load_raw_data()
                             .step_rename_columns()
                            )

In [ ]:
of_course_me_launches_invalid = (of_course_me_launches_raw
                                 .step_replace_missing_values_with_nan()
                                 .step_validate_data()
                                 .step_store_invalid_data()
                                )

In [ ]:
of_course_me_launches_cleaned = (of_course_me_launches_raw
                                 .step_replace_missing_values_with_nan()
                                 .step_remove_not_needed_columns()
                                 .step_remove_rows_with_invalid_ids()
                                 .step_remove_duplicate_rows()
                                 .step_change_datatypes()
                                 .step_store_cleaned_data()
                                )

### home-academy-sections

In [ ]:
datapipe.entity = 'home-academy-sections'

In [ ]:
home_academy_sections_raw = (pd.DataFrame()
                             .step_load_entity_definition()
                             .step_load_raw_data()
                             .step_rename_columns()
                            )

In [ ]:
home_academy_sections_invalid = (home_academy_sections_raw
                                 .step_replace_missing_values_with_nan()
                                 .step_validate_data()
                                 .step_store_invalid_data()
                                )

In [ ]:
home_academy_sections_cleaned = (home_academy_sections_raw
                                 .step_replace_missing_values_with_nan()
                                 .step_remove_not_needed_columns()
                                 .step_remove_rows_with_invalid_ids()
                                 .step_remove_duplicate_rows()
                                 .step_change_datatypes()
                                 .step_store_cleaned_data()
                                )

### traffic-sources

In [ ]:
datapipe.entity = 'traffic-sources'

In [ ]:
traffic_sources_raw = (pd.DataFrame()
                       .step_load_entity_definition()
                       .step_load_raw_data()
                       .step_rename_columns()
                      )

In [ ]:
traffic_sources_invalid = (traffic_sources_raw
                           .step_replace_missing_values_with_nan()
                           .step_validate_data()
                           .step_store_invalid_data()
                          )

In [ ]:
traffic_sources_cleaned = (traffic_sources_raw
                           .step_replace_missing_values_with_nan()
                           .step_remove_not_needed_columns()
                           .step_remove_rows_with_invalid_ids()
                           .step_remove_duplicate_rows()
                           .step_change_datatypes()
                           .step_store_cleaned_data()
                          )

### ga-users

In [ ]:
datapipe.entity = 'ga-users'

In [ ]:
ga_users_raw = (pd.DataFrame()
                .step_load_entity_definition()
                .step_load_raw_data()
                .step_rename_columns()
               )

In [ ]:
ga_users_invalid = (ga_users_raw
                    .step_replace_missing_values_with_nan()
                    .step_validate_data()
                    .step_store_invalid_data()
                   )

In [ ]:
ga_users_cleaned = (ga_users_raw
                    .step_replace_missing_values_with_nan()
                    .step_remove_not_needed_columns()
                    .step_remove_rows_with_invalid_ids()
                    .step_remove_duplicate_rows()
                    .step_change_datatypes()
                    .step_store_cleaned_data()
                   )

### asset-implicit-ratings

In [ ]:
datapipe.entity = 'asset-implicit-ratings'

In [ ]:
asset_implicit_ratings_raw = (pd.DataFrame()
                              .step_load_entity_definition()
                              .step_load_raw_data()
                              .step_rename_columns()
                             )

In [ ]:
asset_implicit_ratings_invalid = (asset_implicit_ratings_raw
                                  .step_replace_missing_values_with_nan()
                                  .step_validate_data()
                                  .step_store_invalid_data()
                                 )

In [ ]:
asset_implicit_ratings_cleaned = (asset_implicit_ratings_raw
                                  .step_replace_missing_values_with_nan()
                                  .step_remove_not_needed_columns()
                                  .step_remove_rows_with_invalid_ids()
                                  .step_remove_duplicate_rows()
                                  .step_change_datatypes()
                                  .step_store_cleaned_data()
                                 )

### lms-items-in-progress

In [ ]:
datapipe.entity = 'lms-assets-in-progress'

In [ ]:
datapipe.definition

,column_name_raw,entity,column_name_clean,column_name_dashboard,unique_identifier,mandatory_field,to_be_removed,datatype,Possible Values,description,Status2,Needed actions


In [ ]:
foo = azure_blob_storage.blob_to_df(container_name='vfu-lms-assets-in-progress-raw',
                                    blob_name='report - 2019-12-11T145951.809.csv')
foo

,STUD_ID,LNAME,FNAME,EMAIL_ADDR,ITEM ID,ITEM TYPE,REV_DTE,REV_NUM,ITEM Market\r\n,Item Title,...,Manager,MGR_EMAIL_ADDR,DELIVERY_TYPE\r\n,ACADEMY,LOCAL_CODE,FUNCTION,BAND,COSTCENTRE,User_Location,Assignment_TYPE
0,10250246,Ford,Graeme,GRAEME.FORD@VODAFONE.COM,6265740,COURSE,5 Jun 2019 05:34,1,VF_GB01,What: Future Capabilities,...,1.03323e+07,BENJAMIN.DAVIES@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,TE,NaN,Local Technology (TE00),H,Transport Operations (GB07017010),At Customer Location (UK617),NaN
1,10250246,Ford,Graeme,GRAEME.FORD@VODAFONE.COM,1760779857,COURSE,23 Aug 2017 12:00,1,VF,TCP and UDP Operation for Cisco CCNA 200-125/1...,...,1.03323e+07,BENJAMIN.DAVIES@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,Local Technology (TE00),Transport Operations (GB07017010),TE,H,Networks,At Customer Location (UK617),NaN
2,10250246,Ford,Graeme,GRAEME.FORD@VODAFONE.COM,1201269208,COURSE,23 Aug 2017 12:00,1,VF,Network Address Translation (NAT) for Cisco CC...,...,1.03323e+07,BENJAMIN.DAVIES@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,Local Technology (TE00),Transport Operations (GB07017010),TE,H,Networks,At Customer Location (UK617),NaN
3,10250246,Ford,Graeme,GRAEME.FORD@VODAFONE.COM,20204605,COURSE,23 Aug 2017 12:00,1,VF,Access Control Lists (ACLs) for Cisco CCNA 200...,...,1.03323e+07,BENJAMIN.DAVIES@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,Local Technology (TE00),Transport Operations (GB07017010),TE,H,Networks,At Customer Location (UK617),NaN
4,10250246,Ford,Graeme,GRAEME.FORD@VODAFONE.COM,1030157725,COURSE,23 Aug 2017 12:00,1,VF,Securing the Switch for Cisco CCNA 200-125/100...,...,1.03323e+07,BENJAMIN.DAVIES@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,Local Technology (TE00),Transport Operations (GB07017010),TE,H,Networks,At Customer Location (UK617),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78301,VF00WM00028494,Agrawal,Chandarani,chanda_rani@rediffmail.com,1638001,COURSE,16 Oct 2017 06:26,1,VF,Perspectives,...,11297230,SANDEEP.SHENOLIKAR@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,HR,NaN,NaN,NaN,NaN,NaN,REQ
78302,VF00WM00028494,Agrawal,Chandarani,chanda_rani@rediffmail.com,2120002,COURSE,22 Nov 2017 05:33,1,VF_GB01,UK Business Continuity Management,...,11297230,SANDEEP.SHENOLIKAR@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,NaN,NaN,HR,NaN,NaN,NaN,NaN
78303,VF00WM00028743,Bateman,Paul,PIBATEMAN@HOTMAIL.CO.UK,2120002,COURSE,22 Nov 2017 05:33,1,VF_GB01,UK Business Continuity Management,...,11306316,ANIL.DEOL@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,NaN,NaN,HR,NaN,NaN,NaN,NaN
78304,VF00WM00028743,Bateman,Paul,PIBATEMAN@HOTMAIL.CO.UK,1638001,COURSE,16 Oct 2017 06:26,1,VF,Perspectives,...,11306316,ANIL.DEOL@VODAFONE.COM,CONTINUOUS ONLINE ACCESS,HR,NaN,NaN,NaN,NaN,NaN,REQ


In [ ]:
lms_assets_in_progress_raw = (pd.DataFrame()
                              .step_load_entity_definition()
                              .step_load_raw_data()
                              .step_rename_columns()
                             )

ValueError: ITEM Market\r\n not present in dataframe columns!

In [ ]:
lms_assets_in_progress_invalid = (lms_assets_in_progress_raw
                                  .step_replace_missing_values_with_nan()
                                  .step_validate_data()
                                  .step_store_invalid_data()
                                 )

In [ ]:
lms_assets_in_progress_raw.columns

Index(['STUD_ID', 'LNAME', 'FNAME', 'EMAIL_ADDR', 'ITEM ID', 'ITEM TYPE',
       'REV_DTE', 'REV_NUM', 'ITEM Market\r\n', 'Item Title', 'Assign Date',
       'Status', 'Credit Hours', 'Duration', 'Grade', 'Scheduled Offering ID',
       'Instructor Name', 'ITEM_SOURCE', 'USER MARKET', 'ORG_ID', 'ORG_DESC',
       'Manager', 'MGR_EMAIL_ADDR', 'DELIVERY_TYPE\r\n', 'ACADEMY',
       'LOCAL_CODE', 'FUNCTION', 'BAND', 'COSTCENTRE', 'User_Location',
       'Assignment_TYPE'],
      dtype='object')

In [ ]:
lms_assets_in_progress_cleaned = (lms_assets_in_progress_raw
                                  .step_replace_missing_values_with_nan()
                                  .step_remove_not_needed_columns()
                                  .step_remove_rows_with_invalid_ids()
                                  .step_remove_duplicate_rows()
                                  .step_format_dates(cols=['completion_date'])
                                  .step_consolidate_lms_item_assignment_types(col='assignment_type')
                                  .step_change_datatypes()
                                  .step_store_cleaned_data()
                                 )

(0, 32)

### user profiles

In [ ]:
azure_blob_storage.list_all_blobs(container_name='vfu-user-profiles-raw')

In [ ]:
user_profiles_old = azure_blob_storage.blob_to_df(container_name='vfu-user-profiles-raw', blob_name='vfu-user-profiles-raw.json')

In [ ]:
user_profiles_weekly = azure_blob_storage.blob_to_df(container_name='vfu-user-profiles-raw', blob_name='vfu-user-profiles-weekly-raw.json')

In [ ]:
user_profiles_weekly[user_profiles_weekly.VFLPLOCALMARKET == 'UK'].EMPLOYEEID.unique()

array([nan])

In [ ]:
user_profiles_weekly.shape

In [ ]:
user_profiles_old[(user_profiles_old.VFUCAREERGROUP.notnull()) & (user_profiles_old.VFLPLOCALMARKET == 'UK')].shape

### step_summarize_if(df)

In [ ]:
import numpy as np
import pandas as pd
import pandas_flavor as pf

In [ ]:
raw_data = {'first_name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
        'last_name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze'], 
        'age': [42, 52, 36, 24, 73], 
        'preTestScore': [4, 24, 31, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
df

,first_name,last_name,age,preTestScore,postTestScore
0,Jason,Miller,42,4,25
1,Molly,Jacobson,52,24,94
2,Tina,Ali,36,31,57
3,Jake,Milner,24,2,62
4,Amy,Cooze,73,3,70


In [ ]:
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)

In [ ]:
[x:= x**2 for x in [1, 2, 3, 4, 5]]

SyntaxError: invalid syntax (<ipython-input-5-77ddb247c5cc>, line 1)

In [ ]:
@pf.register_dataframe_method
def step_summarize_if(df, cond, funs):
    if cond == 'all_numeric':
        cols = df.select_dtypes(include=np.number)
    else:
        df = df.query(cond)
        
    [df[f'{col}_{f.__name__}'] := f(df[col].values) for col in cols for f in funs]
    
    return df

SyntaxError: invalid syntax (<ipython-input-4-041e39f263b2>, line 8)

In [ ]:
df = df.step_summarize_if(cond = 'all_numeric', funs=[np.mean, np.median, np.std])
df

In [ ]:
df

In [ ]:
cond = 'df.select_dtypes(include=np.number)'
pd.eval('df["first_name"]')